# Create Time-Series ML Model

## Load libraries

First, we will load the libraries.

In [1]:
import pandas as pd
import pprint
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

pp = pprint.PrettyPrinter(indent=2)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

## Prepare data

Then we load, and prepare the data.

In [2]:
def sort_by_year_repetition(df):
    # Extract election id components from the index
    df['year'] = df.index.map(lambda x: int(x.split('_')[0][1:5]))  # Assumes year is four digits long
    df['repetitionid'] = df.index.map(lambda x: int(x.split('_')[0][5:]))  # Assumes repetitionid immediately follows year

    # Sort the DataFrame by year and then by repetitionid
    df_sorted = df.sort_values(by=['year', 'repetitionid'])

    # Drop the temporary columns used for sorting
    df_sorted = df_sorted.drop(columns=['year', 'repetitionid'])

    return df_sorted

In [3]:
def add_shifted_columns_grouped(df):
    # Extract 'mundissec' from the index
    df['mundissec'] = df.index.map(lambda x: x.split('_')[1])

    # Identify unique party codes by splitting each column name
    party_codes = set(col.split('_')[-1] for col in df.columns if ('_' in col) and (col.split('_')[-1].isdigit()))

    # Iterate over each party code to create shifted columns group-wise
    for party_code in party_codes:
        # Identify columns for the current party code
        party_columns = [col for col in df.columns if col.endswith(party_code)]
        for col in party_columns:
            # Create a new shifted column name
            shifted_col_name = f"{col}_shifted"
            # Group by 'mundissec' and shift within each group
            df[shifted_col_name] = df.groupby('mundissec')[col].shift(-1)  # Shift within each group

    # Drop the temporary 'mundissec' column after shifting
    df.drop(columns='mundissec', inplace=True)

    return df

In [4]:
df_timeseries = pd.read_pickle("../../data/output/timeseries_2010_2024_6_1_True_True_True_True.pkl")
df_sorted = sort_by_year_repetition(df_timeseries)
df_timeseries = add_shifted_columns_grouped(df_sorted)
df_timeseries.head()

,cens_electoral_percentage_10,cens_electoral_percentage_1001,cens_electoral_percentage_1003,cens_electoral_percentage_1013,cens_electoral_percentage_1015,cens_electoral_percentage_1030,cens_electoral_percentage_1031,cens_electoral_percentage_201914111,cens_electoral_percentage_3000000,cens_electoral_percentage_301,...,cens_electoral_percentage_337_shifted,cens_electoral_percentage_1031_shifted,cens_electoral_percentage_1030_shifted,cens_electoral_percentage_82484191_shifted,cens_electoral_percentage_365_shifted,cens_electoral_percentage_10_shifted,cens_electoral_percentage_431094190_shifted,cens_electoral_percentage_751_shifted,cens_electoral_percentage_999999999_shifted,cens_electoral_percentage_1015_shifted
electionid_mundissec,,,,,,,,,,,,,,,,,,,,,
A20101_08001801001,3.577513,0.0,0.0,0.0,0.0,0.0,17.802385,0.0,0.0,2.555366,...,5.821918,13.698630,0.0,0.0,0.0,4.023973,0.0,0.0,0.870434,0.0
A20101_08001801002,1.992337,0.0,0.0,0.0,0.0,0.0,15.402299,0.0,0.0,1.762452,...,5.490196,9.490196,0.0,0.0,0.0,2.352941,0.0,0.0,0.928105,0.0
A20101_08001801003,3.676471,0.0,0.0,0.0,0.0,0.0,22.610294,0.0,0.0,4.595588,...,5.575540,20.683453,0.0,0.0,0.0,2.158273,0.0,0.0,0.719424,0.0
A20101_08001801004,4.765818,0.0,0.0,0.0,0.0,0.0,21.446179,0.0,0.0,2.793755,...,6.344171,17.287867,0.0,0.0,0.0,3.647898,0.0,0.0,0.660851,0.0
A20101_08001801005,2.558480,0.0,0.0,0.0,0.0,0.0,14.473684,0.0,0.0,1.900585,...,7.211185,9.050773,0.0,0.0,0.0,2.796174,0.0,0.0,1.017905,0.0


In [5]:
df_timeseries.tail(5)

,cens_electoral_percentage_10,cens_electoral_percentage_1001,cens_electoral_percentage_1003,cens_electoral_percentage_1013,cens_electoral_percentage_1015,cens_electoral_percentage_1030,cens_electoral_percentage_1031,cens_electoral_percentage_201914111,cens_electoral_percentage_3000000,cens_electoral_percentage_301,...,cens_electoral_percentage_337_shifted,cens_electoral_percentage_1031_shifted,cens_electoral_percentage_1030_shifted,cens_electoral_percentage_82484191_shifted,cens_electoral_percentage_365_shifted,cens_electoral_percentage_10_shifted,cens_electoral_percentage_431094190_shifted,cens_electoral_percentage_751_shifted,cens_electoral_percentage_999999999_shifted,cens_electoral_percentage_1015_shifted
electionid_mundissec,,,,,,,,,,,,,,,,,,,,,
A20211_43906001002,11.009174,0.0,2.548420,1.630989,0.0,0.0,20.897044,0.0,0.0,2.038736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601001,8.389831,0.0,1.610169,3.644068,0.0,0.0,5.423729,0.0,0.0,3.389831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601002,4.393673,0.0,0.615114,2.372583,0.0,0.0,0.790861,0.0,0.0,3.690685,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601003,7.974138,0.0,2.262931,2.586207,0.0,0.0,6.573276,0.0,0.0,3.232759,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601004,5.937235,0.0,2.120441,2.374894,0.0,0.0,3.307888,0.0,0.0,2.544529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def split_data(df):
    # Identify unique elections from the index
    elections = df.index.map(lambda x: x.split('_')[0]).unique()

    # Latest and penultimate election identifiers
    last_election = elections[-1]
    penultimate_election = elections[-2] if len(elections) > 1 else None

    # Split the DataFrame based on the election identifiers
    new_data = df.loc[df.index.map(lambda x: x.split('_')[0]) == last_election]
    test_data = df.loc[df.index.map(lambda x: x.split('_')[0]) == penultimate_election] if penultimate_election else pd.DataFrame()
    train_data = df.loc[~df.index.map(lambda x: x.split('_')[0]).isin([last_election, penultimate_election])]

    # Columns that are not shifted
    non_shifted_columns = [col for col in df.columns if not col.endswith('_shifted')]
    # Columns that are shifted
    shifted_columns = [col for col in df.columns if col.endswith('_shifted')]

    # Creating new_data, X_test, y_test, X_train, y_train
    new_data = new_data[non_shifted_columns]
    X_test = test_data[non_shifted_columns]
    y_test = test_data[shifted_columns]
    X_train = train_data[non_shifted_columns]
    y_train = train_data[shifted_columns]

    return X_train, y_train, X_test, y_test, new_data

# Test the function
X_train, y_train, X_test, y_test, new_data = split_data(df_timeseries)

# Printing shapes to verify the splits
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')
print(f'New data shape: {new_data.shape}')

X_train shape: (66079, 29), y_train shape: (66079, 21)
X_test shape: (5083, 29), y_test shape: (5083, 21)
New data shape: (5083, 29)


In [7]:
X_train = X_train.astype(float)
y_train = y_train.astype(float)
X_test = X_test.astype(float)
y_test = y_test.astype(float)
new_data = new_data.astype(float)

In [22]:
# Set up the parameters for XGBoost
params_multioutput = {
    "n_estimators": 400,  # Number of boosting rounds
    "max_depth": 10,  # Typically 3-10. Higher values can lead to overfitting.
    "eta": 0.01,  # Learning rate, typically between 0.01 and 0.2
    "objective": "reg:squarederror",  # Regression with squared loss
    "eval_metric": "rmse",  # Root Mean Square Error for evaluation
    "tree_method": "hist",  # Fast histogram optimized approximate greedy algorithm
    "multi_strategy": "multi_output_tree",
    "early_stopping_rounds": 5,
    "reg_alpha": 100,  # L1 regularization term on weights. Increasing this value will make model more conservative. 
    "reg_lambda": 100,  # L2 regularization term on weights. Increasing this value will make model more conservative.
}
# Define the model_multioutput
model_multioutput = XGBRegressor(**params_multioutput)

# Train the model_multioutput
eval_set = [(X_test, y_test)]
model_multioutput.fit(X_train, y_train, eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:4.64218
[1]	validation_0-rmse:4.59022
[2]	validation_0-rmse:4.54875
[3]	validation_0-rmse:4.49814
[4]	validation_0-rmse:4.45868
[5]	validation_0-rmse:4.40950
[6]	validation_0-rmse:4.36123
[7]	validation_0-rmse:4.31384
[8]	validation_0-rmse:4.26983
[9]	validation_0-rmse:4.22430
[10]	validation_0-rmse:4.18718
[11]	validation_0-rmse:4.14277
[12]	validation_0-rmse:4.10618
[13]	validation_0-rmse:4.06231
[14]	validation_0-rmse:4.01865
[15]	validation_0-rmse:3.97832
[16]	validation_0-rmse:3.94329
[17]	validation_0-rmse:3.90358
[18]	validation_0-rmse:3.86121
[19]	validation_0-rmse:3.82184
[20]	validation_0-rmse:3.78093
[21]	validation_0-rmse:3.74324
[22]	validation_0-rmse:3.71099
[23]	validation_0-rmse:3.67420
[24]	validation_0-rmse:3.63585
[25]	validation_0-rmse:3.59858
[26]	validation_0-rmse:3.56773
[27]	validation_0-rmse:3.53123
[28]	validation_0-rmse:3.49660
[29]	validation_0-rmse:3.46081
[30]	validation_0-rmse:3.43240
[31]	validation_0-rmse:3.39761
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eta=0.01, eval_metric='rmse',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy='multi_output_tree',
             n_estimators=400, n_jobs=None, num_parallel_tree=None, ...)

In [23]:
predictions = model_multioutput.predict(X_test, iteration_range=(0, model_multioutput.best_iteration + 1))
if not isinstance(predictions, pd.DataFrame):
    predictions = pd.DataFrame(predictions, index=y_test.index, columns=y_test.columns)

# Initialize a dictionary to store metrics for each column
metrics = {}

# Loop through each column in y_test to calculate metrics
for column in y_test.columns:
    mae = mean_absolute_error(y_test[column], predictions[column])
    mse = mean_squared_error(y_test[column], predictions[column])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test[column], predictions[column])

    # Store metrics in the dictionary
    metrics[column] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R^2': r2}

# Print or return the metrics
for col, values in metrics.items():
    print(f"Metrics for {col}:")
    print(f"  MAE: {values['MAE']}")
    print(f"  MSE: {values['MSE']}")
    print(f"  RMSE: {values['RMSE']}")
    print(f"  R^2: {values['R^2']}\n")

Metrics for cens_electoral_percentage_1001_shifted:
  MAE: 0.7666111234461912
  MSE: 0.59780966304232
  RMSE: 0.7731815201117522
  R^2: 0.0

Metrics for cens_electoral_percentage_86_shifted:
  MAE: 1.7128607204643578
  MSE: 3.870433550191575
  RMSE: 1.9673417471785564
  R^2: -0.573821342331621

Metrics for cens_electoral_percentage_637_shifted:
  MAE: 0.7685917627147725
  MSE: 0.598204293464145
  RMSE: 0.7734366770874944
  R^2: 0.0

Metrics for cens_electoral_percentage_3000000_shifted:
  MAE: 0.7450559554127242
  MSE: 0.5628983719794678
  RMSE: 0.7502655343139972
  R^2: 0.0

Metrics for cens_electoral_percentage_5000000_shifted:
  MAE: 0.8863778505859097
  MSE: 0.7958542090562846
  RMSE: 0.8921066130548997
  R^2: 0.0

Metrics for cens_electoral_percentage_1013_shifted:
  MAE: 1.7033195314864116
  MSE: 4.481394278022045
  RMSE: 2.1169303904526586
  R^2: -0.3940993009747451

Metrics for cens_electoral_percentage_38_shifted:
  MAE: 0.9743876672391462
  MSE: 1.0007442430288085
  RMSE: 1.0

This results are promising, as we got good $R^2$ values for 4 parties. These parties are:
- JxCat (1031): $R^2 = 0.820$
- PSC (6): $R^2 = 0.569$
- CUP (1003): $R^2 = 0.445$
- ERC (10): $R^2 = 0.471$

We've added the election type for each election, as this is an important feature. The results are similar, but slighly worse:

- JxCat (1031): $R^2 = 0.819$
- PSC (6): $R^2 = 0.241$
- CUP (1003): $R^2 = 0.329$
- ERC (10): $R^2 = 0.681$

We've also added the proportion of born abroad and the proportion of different groups of age. The results are similar, but they have slightly improved for PSC, and ERC, and slightly worsened for JxCat and CUP. The results are as follows:

- JxCat (1031): $R^2 = 0.800$
- PSC (6): $R^2 = 0.479$
- CUP (1003): $R^2 = 0.299$
- ERC (10): $R^2 = 0.443$

We've also added the mean income data by census section. The results are again similar, but slightly worse, as the $R^2$ values are very low. That is something strange because the mean income is a very important feature in the prediction of the election results. The results are as follows:

- JxCat (1031): $R^2 = 0.805$
- PSC (6): $R^2 = 0.463$
- CUP (1003): $R^2 = 0.261$
- ERC (10): $R^2 = 0.461$

In [10]:
# Set up the parameters for XGBoost
params = {
    'max_depth': 6,  # Typically 3-10. Higher values can lead to overfitting.
    'eta': 0.1,  # Learning rate, typically between 0.01 and 0.2
    'objective': 'reg:squarederror',  # Regression with squared loss
    'eval_metric': 'rmse',  # Root Mean Square Error for evaluation
    "early_stopping_rounds": 10, 
}
num_boost_round = 200  # Number of boosting rounds

# Define the model
model = XGBRegressor(**params)

# Train the model
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:4.24881
[1]	validation_0-rmse:3.76417
[2]	validation_0-rmse:3.43164
[3]	validation_0-rmse:3.12083
[4]	validation_0-rmse:2.85455
[5]	validation_0-rmse:2.63265
[6]	validation_0-rmse:2.44811
[7]	validation_0-rmse:2.25955
[8]	validation_0-rmse:2.15956
[9]	validation_0-rmse:2.07259
[10]	validation_0-rmse:2.00732
[11]	validation_0-rmse:1.95113
[12]	validation_0-rmse:1.90680
[13]	validation_0-rmse:1.87444
[14]	validation_0-rmse:1.87778
[15]	validation_0-rmse:1.87561
[16]	validation_0-rmse:1.90633
[17]	validation_0-rmse:1.92145
[18]	validation_0-rmse:1.94002
[19]	validation_0-rmse:1.96837
[20]	validation_0-rmse:1.99231
[21]	validation_0-rmse:2.02724
[22]	validation_0-rmse:2.04395
[23]	validation_0-rmse:2.06933


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=10,
             enable_categorical=False, eta=0.1, eval_metric='rmse',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [11]:
# Assuming 'predictions' is a DataFrame or array with the same structure as 'y_test'
predictions = model.predict(X_test)
if not isinstance(predictions, pd.DataFrame):
    predictions = pd.DataFrame(predictions, index=y_test.index, columns=y_test.columns)

# Initialize a dictionary to store metrics for each column
metrics = {}

# Loop through each column in y_test to calculate metrics
for column in y_test.columns:
    mae = mean_absolute_error(y_test[column], predictions[column])
    mse = mean_squared_error(y_test[column], predictions[column])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test[column], predictions[column])

    # Store metrics in the dictionary
    metrics[column] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R^2': r2}

# Print or return the metrics
for col, values in metrics.items():
    print(f"Metrics for {col}:")
    print(f"  MAE: {values['MAE']}")
    print(f"  MSE: {values['MSE']}")
    print(f"  RMSE: {values['RMSE']}")
    print(f"  R^2: {values['R^2']}\n")

Metrics for cens_electoral_percentage_1001_shifted:
  MAE: 0.6633839247359457
  MSE: 0.44008060400414944
  RMSE: 0.6633857128429504
  R^2: 0.0

Metrics for cens_electoral_percentage_86_shifted:
  MAE: 1.1636995691030763
  MSE: 2.550627948396235
  RMSE: 1.5970685484337341
  R^2: -0.03715324122663666

Metrics for cens_electoral_percentage_637_shifted:
  MAE: 0.667268783317268
  MSE: 0.4458231100739962
  RMSE: 0.6676998652643239
  R^2: 0.0

Metrics for cens_electoral_percentage_3000000_shifted:
  MAE: 0.6627719747065653
  MSE: 0.4394082509498331
  RMSE: 0.6628787603701246
  R^2: 0.0

Metrics for cens_electoral_percentage_5000000_shifted:
  MAE: 0.7594773340783391
  MSE: 0.7735676596391464
  RMSE: 0.8795269521959781
  R^2: 0.0

Metrics for cens_electoral_percentage_1013_shifted:
  MAE: 2.5668932155515956
  MSE: 9.552917665266683
  RMSE: 3.0907794591763875
  R^2: -1.971779542971975

Metrics for cens_electoral_percentage_38_shifted:
  MAE: 0.6646655479590067
  MSE: 0.44178056403335
  RMSE: 0

This results are promising, as we got good $R^2$ values for 4 parties. These parties are:
- JxCat (1031): $R^2 = 0.709$
- PSC (6): $R^2 = 0.726$
- CUP (1003): $R^2 = 0.419$
- ERC (10): $R^2 = 0.444$

This means that in some cases the past percentage of votes explains a lot of the future percentage of votes. This is a good sign, as we can use this information to predict the future. But we need to be careful, as this is not always the case.

This parties have in common that they are important, and have consistent data for the past years. Some parties have surprising behaviours that are not explained by the past data.

We've added the election type for each election, as this is an important feature. The results are similar, but slighly worse:

- JxCat (1031): $R^2 = 0.738$
- PSC (6): $R^2 = 0.679$
- CUP (1003): $R^2 = 0.402$
- ERC (10): $R^2 = 0.291$

## Predictions with new data

In [44]:
# Make predictions
new_predictions = model.predict(new_data)
print(f"Predictions for New Data: {new_predictions}")

Predictions for New Data: [[ 2.9525998   0.6624042  14.284147   ...  0.6636474   0.98431915
   0.6673583 ]
 [ 2.6769886   0.6624042  15.805771   ...  0.6636474   1.0255357
   0.6686701 ]
 [ 2.971561    0.6624042  15.143438   ...  0.6636474   0.99451333
   0.6673583 ]
 ...
 [ 1.6247534   0.6624042  16.779013   ...  0.6636474   1.8688223
   0.6630192 ]
 [ 2.7526872   0.6624042  14.180771   ...  0.6636474   0.8404626
   0.6647433 ]
 [ 2.5658808   0.6624042  13.727295   ...  0.6636474   1.8688223
   0.6630192 ]]


In [45]:
# for each column in the predictions, calculate the mean
mean_predictions = new_predictions.mean(axis=0)
mean_predictions

array([ 3.4947915 ,  0.6625365 , 10.9308    ,  3.241869  ,  0.66860753,
        0.94408894,  3.0239863 ,  1.7547996 ,  0.6647472 ,  0.66306853,
        0.7042014 ,  0.66365457,  0.7472769 , 12.07125   ,  0.66242945,
        8.5561695 ,  0.66252285,  0.6624268 ,  0.6646441 ,  1.1047701 ,
        0.70819294], dtype=float32)

In [46]:
mean_predictions = mean_predictions.reshape(1, -1)  # Reshape the array
mean_predictions_df = pd.DataFrame(mean_predictions, columns=y_test.columns)

In [47]:
mean_predictions_df

,cens_electoral_percentage_1003_shifted,cens_electoral_percentage_3000000_shifted,cens_electoral_percentage_6_shifted,cens_electoral_percentage_86_shifted,cens_electoral_percentage_751_shifted,cens_electoral_percentage_1013_shifted,cens_electoral_percentage_301_shifted,cens_electoral_percentage_38_shifted,cens_electoral_percentage_337_shifted,cens_electoral_percentage_82484191_shifted,...,cens_electoral_percentage_1030_shifted,cens_electoral_percentage_1001_shifted,cens_electoral_percentage_10_shifted,cens_electoral_percentage_431094190_shifted,cens_electoral_percentage_1031_shifted,cens_electoral_percentage_201914111_shifted,cens_electoral_percentage_365_shifted,cens_electoral_percentage_637_shifted,cens_electoral_percentage_999999999_shifted,cens_electoral_percentage_1015_shifted
0,3.494792,0.662537,10.9308,3.241869,0.668608,0.944089,3.023986,1.7548,0.664747,0.663069,...,0.663655,0.747277,12.07125,0.662429,8.55617,0.662523,0.662427,0.664644,1.10477,0.708193


In [48]:
total_census = 5754840
total_votes = (mean_predictions_df/100) * total_census
total_votes

,cens_electoral_percentage_1003_shifted,cens_electoral_percentage_3000000_shifted,cens_electoral_percentage_6_shifted,cens_electoral_percentage_86_shifted,cens_electoral_percentage_751_shifted,cens_electoral_percentage_1013_shifted,cens_electoral_percentage_301_shifted,cens_electoral_percentage_38_shifted,cens_electoral_percentage_337_shifted,cens_electoral_percentage_82484191_shifted,...,cens_electoral_percentage_1030_shifted,cens_electoral_percentage_1001_shifted,cens_electoral_percentage_10_shifted,cens_electoral_percentage_431094190_shifted,cens_electoral_percentage_1031_shifted,cens_electoral_percentage_201914111_shifted,cens_electoral_percentage_365_shifted,cens_electoral_percentage_637_shifted,cens_electoral_percentage_999999999_shifted,cens_electoral_percentage_1015_shifted
0,201119.669893,38127.915518,629050.075922,186564.377538,38477.292572,54330.809002,174025.575035,100985.913414,38255.136638,38158.532304,...,38192.257665,43004.589328,694681.102439,38121.754643,492393.851702,38127.130335,38121.604574,38249.206226,63577.748285,40755.371706


## PyCaret

In [108]:
train_data = X_train.copy()
train_data[y_train.columns] = y_train
test_data = X_test.copy()
test_data[y_test.columns] = y_test

# Combine features and target columns for the setup
all_columns = X_train.columns.tolist() + y_train.columns.tolist()

In [109]:
from pycaret.regression import *

for col in y_train.columns:
    print(col)
    train_data = X_train.copy()
    train_data[col] = y_train[col]
    test_data = X_test.copy()
    test_data[col] = y_test[col]

    # Initialize the setup
    reg_setup = setup(
        data=train_data,
        test_data=test_data,
        target=col,
        session_id=123,
        normalize=True,
        transformation=True,
        transform_target=True,
        verbose=False,
    )
    best_model = compare_models()
    print(f"The best model for {col} is: {best_model}")

cens_electoral_percentage_999999999_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.6980
en,Elastic Net,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.1370
dummy,Dummy Regressor,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.2000
llar,Lasso Least Angle Regression,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.1350
omp,Orthogonal Matching Pursuit,1.0200,10.2508,2.6199,-1.1153,0.5147,1.3409,0.1370
ada,AdaBoost Regressor,0.9780,10.1882,2.6024,-1.3978,0.4902,1.1600,0.4840
br,Bayesian Ridge,0.9940,10.1780,2.6267,-2.6629,0.5012,1.2286,0.1360
ridge,Ridge Regression,0.9940,10.1779,2.6267,-2.6658,0.5012,1.2286,0.1400
lr,Linear Regression,0.9940,10.1779,2.6267,-2.6660,0.5012,1.2286,0.7790
lar,Least Angle Regression,1.0006,10.1931,2.6343,-3.0986,0.5043,1.2604,0.1370


The best model for cens_electoral_percentage_999999999_shifted is: Lasso(random_state=123)
cens_electoral_percentage_1031_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,5.6057,77.9349,8.5935,0.1289,0.7357,0.4499,7.8290
et,Extra Trees Regressor,5.5754,77.4537,8.4739,0.0660,0.7280,0.4439,1.8980
xgboost,Extreme Gradient Boosting,5.8696,80.3865,8.7201,0.0418,0.7661,0.4871,0.2600
lightgbm,Light Gradient Boosting Machine,5.8445,79.4175,8.6907,0.0375,0.7445,0.4764,0.4010
huber,Huber Regressor,6.8264,95.9717,9.5813,-0.0749,0.8164,0.6251,0.1930
omp,Orthogonal Matching Pursuit,6.9934,98.7084,9.7488,-0.1031,0.8717,0.7345,0.1620
gbr,Gradient Boosting Regressor,6.8622,94.5992,9.5346,-0.1085,0.8167,0.6143,2.7060
lar,Least Angle Regression,6.9804,100.4649,9.7885,-0.1381,0.8288,0.6342,0.1620
ridge,Ridge Regression,7.0530,101.5133,9.8654,-0.1446,0.8273,0.6359,0.1570
lr,Linear Regression,7.0530,101.5132,9.8654,-0.1446,0.8273,0.6359,0.1490


The best model for cens_electoral_percentage_1031_shifted is: RandomForestRegressor(n_jobs=-1, random_state=123)
cens_electoral_percentage_86_shifted


...

The best model for cens_electoral_percentage_1000_shifted is: XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=123, ...)
cens_electoral_percentage_6_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,3.3378,24.4317,4.5765,0.4880,0.4780,0.3478,0.5090
et,Extra Trees Regressor,3.3087,24.2739,4.5628,0.4850,0.4801,0.3438,2.8930
rf,Random Forest Regressor,3.3751,24.9835,4.6577,0.4681,0.4908,0.3549,10.7590
xgboost,Extreme Gradient Boosting,3.4283,25.2413,4.6817,0.4634,0.4923,0.3605,0.3210
gbr,Gradient Boosting Regressor,3.5491,25.8033,4.7539,0.4549,0.4896,0.3750,3.3590
knn,K Neighbors Regressor,3.6589,28.7396,5.0306,0.3548,0.5361,0.3895,0.8460
br,Bayesian Ridge,4.2142,32.0240,5.4242,0.3106,0.5788,0.4567,0.2390
lr,Linear Regression,4.2144,32.0276,5.4245,0.3105,0.5788,0.4568,0.2320
ridge,Ridge Regression,4.2144,32.0272,5.4245,0.3105,0.5788,0.4568,0.2410
lar,Least Angle Regression,4.2383,32.3168,5.4457,0.3053,0.5821,0.4591,0.2360


The best model for cens_electoral_percentage_6_shifted is: LGBMRegressor(n_jobs=-1, random_state=123)
cens_electoral_percentage_693_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,0.0170,0.0079,0.0474,-0.0053,0.0303,nan,0.2870
lightgbm,Light Gradient Boosting Machine,0.0164,0.0078,0.0450,-0.0053,0.0287,nan,0.4800
gbr,Gradient Boosting Regressor,0.0390,0.0206,0.0939,-0.3063,0.0565,nan,2.8410
lasso,Lasso Regression,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.1920
dummy,Dummy Regressor,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.1860
llar,Lasso Least Angle Regression,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.1970
en,Elastic Net,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.2070
huber,Huber Regressor,0.3227,0.6747,0.4395,-1.6295,0.2087,nan,0.3740
par,Passive Aggressive Regressor,0.6805,2.5268,0.8944,-1.7321,0.3626,nan,0.2140
br,Bayesian Ridge,0.3558,0.7709,0.4840,-1.7416,0.2343,nan,0.2190


The best model for cens_electoral_percentage_693_shifted is: XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=123, ...)
